# 🧠 Advanced RAG Agent — Documentation Interne
Ce notebook construit un agent RAG capable de répondre à des questions sur une documentation interne, en utilisant FAISS, LangChain LCEL, et Mistral-7B.

In [ ]:
# Installation des bibliothèques nécessaires:
!pip install -q langchain transformers sentence-transformers faiss-cpu accelerate torch langchain_community langchain-huggingface langchain-core
# pour utilisé ChromaDB:
# pip install -q chromadb

In [ ]:
# Téléchargement des dépendances Python:
import os
import torch
from langchain_community.vectorstores import FAISS 
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain.llms import HuggingFacePipeline 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("Installations et imports terminés.")

In [ ]:
# CONFIGURATION DES COMPOSANTS RAG :

FILE_PATH = "documentation_interne.txt"

# Chargement et Division des données (Chunking):
try:
    with open(FILE_PATH, 'r', encoding='utf-8') as f:
        document_content = f.read()
except FileNotFoundError:
    print(f"ERREUR: Le fichier {FILE_PATH} n'a pas été trouvé. Veuillez le téléverser dans Colab.")
    exit()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)
docs = text_splitter.create_documents([document_content])
print(f"Document divisé en {len(docs)} segments (chunks).")

# EMBEDDING (Hugging Face Local et Gratuit):
embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'} 
)

# CRÉATION DU VECTOR STORE (Mémoire RAG):
vector_store = FAISS.from_documents(docs, embedding_function)
rag_retriever = vector_store.as_retriever(search_kwargs={"k": 3})
print("✅ Base de données FAISS (RAG) créée avec succès.")

In [ ]:

%%writefile rag_chain.py
# Charger Mistral 7B Instruct depuis Hugging Face:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Utiliser le pipeline Transformers (GPU de Colab)
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 512,
        # 'device' est retiré ici pour éviter le conflit
    },
    device=0,
    model_kwargs={"torch_dtype": torch.bfloat16} 
)
print(f"✅ LLM Agent (Mistral-7B-Instruct) chargé depuis Hugging Face sur Colab.")


# --- CONSTRUCTION DE LA CHAÎNE RAG DIRECTE (LCEL) ---

RAG_PROMPT = ChatPromptTemplate.from_template("""
Tu es un assistant IA professionnel. Utilise UNIQUEMENT le contexte fourni ci-dessous pour répondre à la question. 
Si la réponse n'est pas dans le contexte, tu dois répondre de manière polie : 'Je suis désolé, cette information spécifique n'est pas disponible dans ma documentation interne.'

--- CONTEXTE ---
{context}
--- FIN CONTEXTE ---

Question: {question}
""")

# Assemblage de la Chaîne RAG LCEL
rag_chain = (
    RunnablePassthrough.assign(
        context= (lambda x: x['question']) | rag_retriever 
    )
    | RAG_PROMPT
    | llm # Le LLM HuggingFacePipeline est utilisé ici
    | StrOutputParser()
)

# --- TESTS DE VALIDATION ---

print("\n" + "="*50)
print("TEST DE LA CHAÎNE RAG DIRECTE SUR COLAB (Validation effectuée)")
print("="*50)

# Test 1: Question Interne (utilisation du RAG pour répondre)
print("\n--- TEST 1: Question Interne ---")
question_interne = "Combien de temps faut-il pour finaliser le traitement d'un remboursement ?"
print(f"Question : {question_interne}")
response_interne = rag_chain.invoke({"question": question_interne})
print(f"\nRÉPONSE RAG : {response_interne}")

# Test 2: Question Généraliste (DOIT répondre 'Information non disponible')
print("\n--- TEST 2: Question Générale ---")
question_generale = "Quel est le plus grand désert du monde ?"
print(f"Question : {question_generale}")
response_generale = rag_chain.invoke({"question": question_generale})
print(f"RÉPONSE RAG : {response_generale}")

In [ ]:
%%writefile app.py
from fastapi import FastAPI
from pydantic import BaseModel
from rag_chain import rag_chain  # Assure-toi que rag_chain est exporté dans rag_chain.py

app = FastAPI()

class Query(BaseModel):
    question: str

@app.post("/ask")
async def ask_question(query: Query):
    """
    Pose une question à l'agent RAG.
    - **question**: Texte de la question à poser
    - **return**: Réponse générée par le modèle, basée sur la documentation interne
    """
    response = rag_chain.invoke({"question": query.question})
    return {"answer": response}

@app.get("/health")
def health_check():
    return {"status": "ok"}


In [ ]:
!pip install pyngrok
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 &

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print("🔗 URL publique de ton API :", public_url)

In [ ]:
!ngrok config add-authtoken #YOUR_NGROK_AUTH_TOKEN